In [1]:
import pandas as pd
import numpy as np
import os
import sys
import polars as pl
import json
from joblib import Parallel, delayed
import deepchem
from tqdm import tqdm

# Configure Polars 
cfg = pl.Config()
cfg.set_tbl_rows(20)
cfg.set_tbl_cols(50)
from sklearn.model_selection import StratifiedGroupKFold, GroupKFold, KFold
import gc

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/opt/conda/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dep

In [2]:
PERCENTILES = [0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.975, 0.99]

In [3]:
with open('/home/dangnh36/datasets/competitions/leash_belka/processed/meta/building_blocks.json', 'r') as f:
    bbs_meta = json.load(f)

[print(k, '-->', len(v)) for k, v in bbs_meta.items()]

train_bbs --> 1145
train_bb1s --> 271
train_bb2s --> 693
train_bb3s --> 872
test_bb1s --> 341
test_bb2s --> 1140
test_bb3s --> 1389
test_bbs --> 2110
all_bbs --> 2110


[None, None, None, None, None, None, None, None, None]

In [4]:
with open('/home/dangnh36/datasets/competitions/leash_belka/processed/meta/scaffolds.json', 'r') as f:
    scaffolds = json.load(f)
print(len(scaffolds))

5971685


In [5]:
train_scaffolds = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/train_scaffold.csv').collect()
train_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/train_v2.csv')\
    .select(
        pl.col('molecule'),
        pl.col('bb1', 'bb2', 'bb3').cast(pl.UInt16),
        pl.col('BRD4', 'HSA', 'eSH').cast(pl.UInt8),
        scaffold_idx = train_scaffolds['scaffold_idx'],
    )\
    .collect()
print(train_df.estimated_size('mb'))
train_df

8601.91998577118


molecule,bb1,bb2,bb3,BRD4,HSA,eSH,scaffold_idx
str,u16,u16,u16,u8,u8,u8,i64
"""C#CCOc1ccc(CNc…",1640,1653,765,0,0,0,4283326
"""C#CCOc1ccc(CNc…",1640,1653,205,0,0,0,4486646
"""C#CCOc1ccc(CNc…",1640,1653,1653,0,0,0,1015728
"""C#CCOc1ccc(CNc…",1640,1653,146,0,0,0,5301385
"""C#CCOc1ccc(CNc…",1640,1653,439,0,0,0,5301385
"""C#CCOc1ccc(CNc…",1640,1653,196,0,0,0,5301385
"""C#CCOc1ccc(CNc…",1640,1653,253,0,0,0,5301385
"""C#CCOc1ccc(CNc…",1640,1653,1219,0,0,0,5301385
"""C#CCOc1ccc(CNc…",1640,1653,604,0,0,0,543172


In [86]:
test_scaffolds = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/test_scaffold.csv').collect()
test_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/test_v2.csv')\
    .select(
        pl.col('id','molecule'),
        pl.col('bb1', 'bb2', 'bb3').cast(pl.UInt16),
        pl.col('protein'),
        scaffold_idx = test_scaffolds['scaffold_idx']
    ).group_by('molecule').first().collect()
print(test_df.estimated_size('mb'))
test_df

84.21934127807617


molecule,id,bb1,bb2,bb3,protein,scaffold_idx
str,i64,u16,u16,u16,str,i64
"""Cn1ncc2c(C(=O)…",295336005,304,645,1201,"""BRD4""",1024626
"""CC(C)(C#N)c1cc…",295987102,398,1173,979,"""HSA""",5417893
"""COc1nc(Br)ccc1…",296231759,511,1880,1041,"""BRD4""",204272
"""O=C(N[Dy])c1cc…",296406854,1798,801,1792,"""BRD4""",3756277
"""CS(=O)(=O)c1cc…",295359211,1413,50,1203,"""sEH""",5785756
"""O=C(N[Dy])c1cc…",296478917,2036,1944,329,"""BRD4""",216077
"""CC(C)(C)OC(=O)…",295300740,349,1106,16,"""BRD4""",724788
"""CCCCCC1CCC(C(=…",296862284,58,791,1764,"""HSA""",1369803
"""O=C(C[C@@H](Cc…",296593445,1129,828,677,"""BRD4""",4656606


In [8]:
all_bbs = bbs_meta['all_bbs']
len(all_bbs)

2110

In [9]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

In [50]:
conn = np.zeros((len(all_bbs), len(all_bbs)), dtype = np.uint8)
conn

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [51]:
for row in tqdm(train_df.iter_rows(named=True)):
    i1, i2, i3 = sorted([row['bb1'], row['bb2'], row['bb3']])
    conn[i1, i2] = 1
    conn[i1, i3] = 1
    conn[i2, i3] = 1

conn.sum()

98415610it [03:23, 483619.78it/s]


601373

In [56]:
for row in tqdm(test_df.iter_rows(named=True)):
    i1, i2, i3 = sorted([row['bb1'], row['bb2'], row['bb3']])
    conn[i1, i2] = 1
    conn[i1, i3] = 1
    conn[i2, i3] = 1

conn.sum()

878022it [00:01, 467440.44it/s]


793416

In [57]:
graph = csr_matrix(conn)
n_components, labels = connected_components(csgraph=graph, directed=False, return_labels=True)
n_components

4

In [58]:
np.unique(labels)

array([0, 1, 2, 3], dtype=int32)

In [91]:
bb_cluster = {}

for c in range(n_components):
    cluster_idxs, = np.where(labels == c)
    # print(cluster_idxs)
    bb_cluster[c] = cluster_idxs.tolist()
    print(c, '-->', len(cluster_idxs))

0 --> 1145
1 --> 859
2 --> 53
3 --> 53


In [92]:
with open('/home/dangnh36/datasets/competitions/leash_belka/processed/meta/building_blocks_cluster.json', 'w') as f:
    json.dump(bb_cluster, f)

In [87]:
for cluster_idx in bb_cluster:
    cluster = bb_cluster[cluster_idx]
    num_samples = test_df.filter(pl.col('bb1').is_in(cluster) | pl.col('bb2').is_in(cluster) | pl.col('bb2').is_in(cluster)).shape[0]
    pct = round(num_samples / test_df.shape[0] * 100, 2)
    print(f'cluster={cluster_idx}, {len(cluster)} building blocks, {num_samples} samples ({pct} %)')

cluster=0, 1145 building blocks, 369039 samples (42.03 %)
cluster=1, 859 building blocks, 486390 samples (55.4 %)
cluster=2, 53 building blocks, 11271 samples (1.28 %)
cluster=3, 53 building blocks, 11322 samples (1.29 %)


In [94]:
53 * 52 * 51 / 6 / 2

11713.0

In [83]:
test_scaffolds = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/test_scaffold.csv').collect()
test_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/test_v2.csv')\
    .select(
        pl.col('id','molecule'),
        pl.col('bb1', 'bb2', 'bb3').cast(pl.UInt16),
        pl.col('protein'),
        scaffold_idx = test_scaffolds['scaffold_idx']
    ).collect()
print(test_df.estimated_size('mb'))
test_df

160.23621368408203


id,molecule,bb1,bb2,bb3,protein,scaffold_idx
i64,str,u16,u16,u16,str,i64
295246830,"""C#CCCC[C@H](Nc…",1989,409,409,"""BRD4""",2217250
295246831,"""C#CCCC[C@H](Nc…",1989,409,409,"""HSA""",2217250
295246832,"""C#CCCC[C@H](Nc…",1989,409,409,"""sEH""",2217250
295246833,"""C#CCCC[C@H](Nc…",1989,409,1012,"""BRD4""",602641
295246834,"""C#CCCC[C@H](Nc…",1989,409,1012,"""HSA""",602641
295246835,"""C#CCCC[C@H](Nc…",1989,409,1012,"""sEH""",602641
295246836,"""C#CCCC[C@H](Nc…",1989,409,1722,"""BRD4""",4502748
295246837,"""C#CCCC[C@H](Nc…",1989,409,1722,"""HSA""",4502748
295246838,"""C#CCCC[C@H](Nc…",1989,409,1722,"""sEH""",4502748


In [84]:
test_df.shape

(1674896, 7)

In [85]:
for cluster_idx in bb_cluster:
    cluster = bb_cluster[cluster_idx]
    num_samples = test_df.filter(pl.col('bb1').is_in(cluster) | pl.col('bb2').is_in(cluster) | pl.col('bb2').is_in(cluster)).shape[0]
    pct = round(num_samples / test_df.shape[0] * 100, 2)
    print(f'cluster={cluster_idx}, {len(cluster)} building blocks, {num_samples} samples ({pct} %)')

cluster=0, 1145 building blocks, 1107117 samples (66.1 %)
cluster=1, 859 building blocks, 500000 samples (29.85 %)
cluster=2, 53 building blocks, 33813 samples (2.02 %)
cluster=3, 53 building blocks, 33966 samples (2.03 %)
